### Using Boruta on the Madalon Data Set
Author: [Mike Bernico](mike.bernico@gmail.com)

This example demonstrates using Boruta to find all relevant features in the Madalon dataset, which is an artificial dataset used in NIPS2003 and cited in the [Boruta paper](https://www.jstatsoft.org/article/view/v036i11/v36i11.pdf)

This dataset has 2000 observations and 500 features.  We will use Boruta to identify the features that are relevant to the classification task.





In [1]:
# Installation
#!pip install boruta

In [2]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy

In [3]:
def load_data():
    # URLS for dataset via UCI
    train_data_url='https://archive.ics.uci.edu/ml/machine-learning-databases/madelon/MADELON/madelon_train.data'
    train_label_url='https://archive.ics.uci.edu/ml/machine-learning-databases/madelon/MADELON/madelon_train.labels'
        
    
    X_data = pd.read_csv(train_data_url, sep=" ", header=None)
    y_data = pd.read_csv(train_label_url, sep=" ", header=None)
    data = X_data.ix[:,0:499]
    data['target'] = y_data[0] 
    return data

In [4]:
data = load_data()

In [5]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,target
0,485,477,537,479,452,471,491,476,475,473,...,481,477,485,511,485,481,479,475,496,-1
1,483,458,460,487,587,475,526,479,485,469,...,478,487,338,513,486,483,492,510,517,-1
2,487,542,499,468,448,471,442,478,480,477,...,481,492,650,506,501,480,489,499,498,-1
3,480,491,510,485,495,472,417,474,502,476,...,480,474,572,454,469,475,482,494,461,1
4,484,502,528,489,466,481,402,478,487,468,...,479,452,435,486,508,481,504,495,511,1


In [6]:
y=data.pop('target')
X=data.copy()

Boruta conforms to the sklearn api and can be used in a Pipeline as well as on it's own. Here we will demonstrate stand alone operation.

First we will instantiate an estimator that Boruta will use.  Then we will instantiate a Boruta Object.

In [7]:
rf = RandomForestClassifier(n_jobs=-1, class_weight='auto', max_depth=7)
# define Boruta feature selection method
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2)

Once built, we can use this object to identify the relevant features in our dataset.

In [8]:
feat_selector.fit(X,y)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	500
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	500
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	500
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	500
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	500
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	500
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	500
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	40
Rejected: 	460
Iteration: 	9 / 100
Confirmed: 	20
Tentative: 	20
Rejected: 	460
Iteration: 	10 / 100
Confirmed: 	20
Tentative: 	20
Rejected: 	460
Iteration: 	11 / 100
Confirmed: 	20
Tentative: 	20
Rejected: 	460
Iteration: 	12 / 100
Confirmed: 	21
Tentative: 	12
Rejected: 	467
Iteration: 	13 / 100
Confirmed: 	21
Tentative: 	12
Rejected: 	467
Iteration: 	14 / 100
Confirmed: 	21
Tentative: 	12
Rejected: 	467
Iteration: 	15 / 100
Confirmed: 	21
Tentative: 	12
Rejected: 	467
Iteration: 	16 / 100
Confirmed: 	2

Boruta has confirmed only a few features as useful.   When our run ended, Boruta was undecided on 2 features.   '

We can interrogate .support_ to understand which features were selected.   .support_ returns an array of booleans that we can use to slice our feature matrix to include only relevant columns.   Of course, .transform can also be used, as expected in the scikit API.

In [9]:
# check selected features
print(feat_selector.support_)
#select the chosen features from our dataframe.
selected = X.ix[:,feat_selector.support_]
print ("")
print ("Selected Feature Matrix Shape")
print (selected.shape)

[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False  True False False False False False False False
 False False False False False False False False False False False False
  True False False False False False False False False False False False
 False False False False  True False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False  True False False
 False False False False False False False False False False False False
 False False False False False False False False  True False False False
 False False False False False False False False False False False False
 False False False False False False False False False  True False False
 False False False False False False False False Fa

We can also interrogate the ranking of the unselected features with .ranking_

In [10]:
feat_selector.ranking_

array([445, 219, 176, 405,   3, 361, 231, 472, 359, 419,   2, 302,  78,
        93, 354, 396, 216, 116,  42, 434, 386, 267, 430, 256, 107, 274,
       133, 438,   1, 276, 389, 240,  39, 312, 290,  85, 270, 360,  31,
       462, 448, 284, 130,  44,  11, 394, 135, 282,   1,  80, 370, 107,
       168, 440,  65,   8,  16, 206, 280, 192, 146, 348, 350, 450,   1,
       164, 182, 390, 333, 372, 370, 280, 202,  28, 151,  75, 415, 243,
       228, 104, 305,  71, 258, 233, 221, 107, 453, 412, 147, 202, 478,
       204, 415, 358, 364, 139, 449, 137, 326, 102, 410, 400, 263, 217,
       131,   1,  71, 240, 337, 175, 309, 127, 392, 221,  83, 305,  57,
       126, 298,  94, 457, 255, 226, 267, 214, 152, 218, 186,   1,  38,
       188,  89, 301, 444, 308, 361,  25,   8, 379, 112,  97, 287, 405,
       425, 137, 214, 432,  46, 317,  34, 180, 253, 157,   1, 462, 101,
        92, 302, 174, 190, 299,  54, 433, 428,  82, 267, 472, 377, 474,
       427, 149, 202, 186, 462, 407, 150, 310,  76, 400, 191,  4